# Import Packages

In [ ]:
# Version 2.0 : uses waveforms as a function of time ... 
import sys
import os

PATH_with_functions = '/home/ardhuin/TOOLS/OPTOOLS/PYTHON/'
sys.path.append(PATH_with_functions)

import glob
import xarray as xr
import numpy as np
import pandas as pd

#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
import scipy.interpolate as spi
import scipy.integrate as spint
from scipy.ndimage import gaussian_filter, correlate
from scipy.signal import hilbert,hilbert2,fftconvolve

from functions_cfosat_env import *
from surface_simulation_functions import *
from altimetry_waveforms_functions import *
from matplotlib.dates import DateFormatter
# --- plotting and interactive stuff ----------------------
import matplotlib.pyplot as plt
# from matplotlib.ticker import AutoMinorLocator, FixedLocator

#from Envelope_convolution_functions import *

cNorm = mcolors.Normalize(vmin=0, vmax=2)
jet = plt.get_cmap('jet')
mpl.rcParams.update({'figure.figsize':[10,6],'axes.grid' : True,'font.size': 14,'savefig.facecolor':'white'})

cmap0 = 'viridis'

## Define waveforms with wave group effect

In [ ]:
clight = 299792458
Rearth = 6370*1e3

# --- satellite parameters ----------------------------
v_sat = 7*1e3 # satellite velocity in m/s
Altis = np.array([519*1e3, 781*1e3, 800*1e3, 891*1e3, 1340*1e3])
namesAltis = ['CFOSAT', 'SARAL', 'ENVISAT', 'SWOT', 'Jason3']
BW = np.array([(400*1e6),(500*1e6),(320*1e6),(320*1e6),(320*1e6)])
theta3dB=np.array([(1.6),(0.61),(1.28),(1.28),(1.28)])*np.pi/180.

ialti = 0

Gamma = 0.5 * (1.0/np.log(2.0))*np.sin(theta3dB[ialti])*np.sin(theta3dB[ialti])
Gamma = 1.
alti_sat = Altis[ialti]
bandwidth = BW[ialti]       # bandwidth in Hz (note that CFOSAT/SWIM is oversampled to a 400 MHz band)
nameSat = namesAltis[ialti]
tau=1E9/bandwidth           # time interval in nanoseconds

dr = clight * 1/(2*bandwidth)
nominal_tracking_gate=70

# --- radius to compute waveforms --------------
Hs=10.0095
DiamChelton = calc_footprint_diam(Hs,Rorbit = alti_sat, pulse_width=1/bandwidth)
radi0=DiamChelton # size of diameter of footprint in pixel
print('Chelton footprint diameter in km:',DiamChelton*0.001)
nominal_tracking_gate=70
min_range_fit=10
range_offset = nominal_tracking_gate*dr  # number of meters above nadir sea level included in waveform

ranges = np.arange(0,127*dr,dr)    # remember to increase 35 to higher values if needed
rlim = np.sqrt((radi0/2)**2+(alti_sat)**2)-alti_sat+range_offset
max_range_fit=np.argmax(ranges[ranges<=rlim])
print('Maximum range used for retracking:',max_range_fit)


wf_model='wf_erf4D'
noise=1E-3
wf_fit0=wf_eval(ranges,np.array((0,Hs,1.,0,0.)),clight,wf_model,noise=noise,alti_sat=alti_sat,\
                            tau=tau,nominal_tracking_gate=nominal_tracking_gate,mispointing=0.,theta3dB=theta3dB)
wf_fit=wf_eval(ranges,np.array((0,Hs,1.,0.075,0.)),clight,wf_model,noise=noise,alti_sat=alti_sat,\
                            tau=tau,nominal_tracking_gate=nominal_tracking_gate,mispointing=0.,theta3dB=theta3dB)
fig,axs=plt.subplots(1,2,figsize=(14,6))
ax=axs[0]
line1=ax.plot(ranges[min_range_fit:max_range_fit],wf_fit0[min_range_fit:max_range_fit],color='k',label='erf2D')
line2=ax.plot(ranges[min_range_fit:max_range_fit],wf_fit[min_range_fit:max_range_fit],color='r',label='erf4D',linestyle='dotted')
ax.set_xlabel('range (m)')
ax.set_ylabel('waveform')
ax=axs[1]
line1=ax.plot(ranges[min_range_fit:max_range_fit],wf_fit[min_range_fit:max_range_fit]-wf_fit0[min_range_fit:max_range_fit],color='k',label='erf2D')
ax.set_xlabel('range (m)')
ax.set_ylabel('perturbation')

waveforms=np.expand_dims(np.expand_dims(wf_fit, 0),0)
Hs_retrack,ze_retrack,Pu,dar,R0r,di_retrack=retrack_waveforms(waveforms,ranges,max_range_fit,clight,tau=tau,\
                                    #min_method='gridsearch',wf_model=wf_model,costfun='LS',\
                                    min_method='pyramid4',wf_model=wf_model,costfun='LS',mispointing=[0.],\
                                    #min_method='Nelder-Mead',wf_model=wf_model,costfun='LS',mispointing=[0.],\
                                    alti_sat=alti_sat,nominal_tracking_gate=nominal_tracking_gate,min_range_fit=10)

ip=0
jp=0
print('Retracking result:',Hs_retrack[ip,jp],ze_retrack[ip,jp],di_retrack[ip,jp],dar[ip,jp],R0r[ip,jp])

